# PEA resource estimate

1. Basic resource estimate equation: from circuit input and backend properties and options estimate resource usage
    - resource estimate below is quite off from the qpu runtime
    - can we even get a decent estimate on runtime of a circuit??
2. Identify what we need to change in this notebook for the working session
    - how do the options in the estimator influence the protocol?
    - if they try running their circuit of interest and we get back the information on the results/metadata, what can we learn?
    - Modify Saki's notebook to take a user defined circuit and do PEA on it with pre-defined options (they can change them if they want) and collect info on execution
3. Review survey questions, ensuring that we get the information we need on enabled projects (what do we think we need to know?)
    - input circuits
    - input metadata 
    - output metadata



## Theory on resource estimate

1. characterization protocol for each unique layer: resources needed in terms of input/options/backend props
    - 9 basis to characterize all 2Q Paulis
    - Apply different numbers of 2Q Clifford (CX, CZ, ECR) (depth specified by user -- could estimate given backend properties)
    - Specify number of twirls for each cycle benchmarking experiment (rule of thumb is about 30-100)
    - Specify number of shots (20 shots per circuit, more or less gives less information per shot)

        
`TOTAL number of shots =  number_unique_layers*9_basis*number_cycle_depths*number_randomizations*shots_per_randomization` use rep_delay_time for rough time estimate (backend_prop reports rep_delay)


2. mitigation protocol: resources needed in terms of input/options/backend props: once we know the layer noise (user know the gamma per circuit):
    - number of randomizations (independent of total gamma of circuit)
    - number of shots per randomization
    - number of noise gains (generally 3+) -- picking the noise gains is non-trivial, could estimate this

`TOTAL number of shots = number_of_noise_gains*number_of_randomizations*number_of_shots_per_randomization`



#### Notes:
- Exclude trex estimation from Estimator job for now
- Where is the classical overhead coming from?


#### Estimator Options

In [ ]:
# # Base option formatting
# # Builtin resilience settings for ZNE
# options = {
#     # Builtin resilience settings for ZNE
#         "resilience": {
#         "measure_mitigation": True,
#         "zne_mitigation": True,
#         "zne": {"noise_factors": noise_factors},

        
#         # PEA noise model configuration
#         "layer_noise_learning": {
#             "max_layers_to_learn": 10, # HOW DO YOU PICK WHICH LAYERS TO LEARN (maybe using backend props to estimate?)
#             "layer_pair_depths": learning_pair_depths = #[0, 4, 8, 16, 32, 48, 1300], # CHECK DEFAULTS AND GIVE PLAUSIBEL ESTIMATE FROM BACKEND PROPS
#             "shots_per_randomization": shots_per_randomization, #20 JUSTIFIED BY ANDREW WACK
#             "num_randomizations": num_randomizations_learning, #30-100 CLARIFY WITH IAN
#         },
#     },
    
#     # Randomization configuration
#     "twirling": {
#         "num_randomizations": num_randomizations,
#         "shots_per_randomization": shots_per_randomization,
#         "strategy": "all",
#     },

#     # Experimental settings for PEA method
#     "experimental": {
#         # # Just in case, disable any further qiskit transpilation not related to twirling / DD
#         # "skip_transpilation": True,

#         # Execution configuration
#         "execution": {
#             "max_pubs_per_batch_job": max_batch_circuits,
#             "fast_parametric_update": True,
#         },

#         # Error Mitigation configuration
#         "resilience": {
#             # ZNE Configuration
#             "zne": {
#                 "amplifier": "pea",
#                 "return_all_extrapolated": True,
#                 "return_unextrapolated": True,
#                 "extrapolated_noise_factors": [0, 0.25, 0.5, 0.75] + noise_factors,
#                 # MAX NUMBER OF UNIQUE LAYERS TO MITIGATE/MAX NUMBER OF TOTAL SHOTS?
#                 # OTHER OPTIONS?
#             }
#         }
#     }
# }


#### Estimating depths for cycle benchmarking (noise learning)

In [ ]:
# 1. calcualte number of CXs in least noisy layer

# 2. get cx fidelity from reported ELPG (or use reported value to get average CX error)
# for el in backend_real.properties().general:

#     if el.name[:2] == 'lf' and el.name[3:] == str(num_qubits):
#         lf = el.value # layer fidelity
#         print('layer fidelity', lf)
#         elpg = 1-lf**(1/(num_qubits-1)) # elpg
#         # print('ELPG', elpg)
#         fid_cx = 1 - elpg

# 3. Calculate max number of CXs (or layer reps) to reach 0 fidelity for the least noisy unique layer and derive max number of reps of that layer

# 4. pick a logarithmically spaced interval of depths [0, max_layer_depths] with X number of points in the interval




#### Estimating noise gains

In [ ]:

#noise learning gives the total gamma of the circuit = gamma_circ
#num_2q_ops number of gates in the circuit
gamma_cx = gamma_circ/num_2q_ops
# covert to 2q fidelity
fid_cx = sqrt(1/gamma_cx)

#number of ops before fidelity drops to zero
max_num_cx = int(1/(1-fid_cx)) 

#maximum noise gain is the ratio between the max number of cx 
max_noise_gain = max_num_cx/num_2q_ops

noise_gains = uniform(1, max_noise_gain, number of noise gains) # pick three values of noise gains between 1 and the max uniformly
